![dphi banner](https://dphi-courses.s3.ap-south-1.amazonaws.com/Datathons/dphi_banner.png)

# **[Data Sprint #27: Connect-4](https://dphi.tech/practice/challenge/64)**
Based on code by [Manish KC](https://dphi.tech/notebooks/957/manish_kc_06/data-sprint-27-connect-4)

## Import Libraries

In [ ]:
# To measure execution time of code
!pip install ipython-autotime
 
%load_ext autotime

time: 2.26 ms (started: 2021-03-15 13:05:47 +00:00)


In [ ]:
import pandas as pd
import numpy as np

time: 1.41 ms (started: 2021-03-15 13:05:47 +00:00)


In [ ]:
# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

time: 1.48 ms (started: 2021-03-15 13:05:47 +00:00)


In [ ]:
!pip install pycaret

     |████████████████████████████████| 266kB 5.7MB/s 
     |████████████████████████████████| 1.8MB 20.0MB/s 
     |████████████████████████████████| 14.2MB 304kB/s 
     |████████████████████████████████| 6.8MB 42.9MB/s 
     |████████████████████████████████| 245kB 39.3MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 1.7MB 41.4MB/s 
     |████████████████████████████████| 276kB 41.7MB/s 
     |████████████████████████████████| 215kB 34.8MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 1.1MB 41.9MB/s 
     |████████████████████████████████| 163kB 39.3MB/s 
     |████████████████████████████████| 153kB 41.0MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 1.1MB 38.2MB/s 
     |████████████████████████████████| 3.2MB 37.4MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
 

In [ ]:
import pycaret

time: 5.66 ms (started: 2021-03-15 13:06:19 +00:00)


## Load the Training Data

In [ ]:
train_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/connect-4/train_set_label.csv")

time: 598 ms (started: 2021-03-15 13:06:19 +00:00)


## Exploratory Data Analysis

In [ ]:
# View the first few rows of the training data
train_data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,b,b,b,b,b,b,o,o,x,b,b,b,x,x,o,x,b,b,b,b,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,loss
1,b,b,b,b,b,b,o,b,b,b,b,b,x,o,o,b,b,b,x,b,b,b,b,b,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,loss
2,x,x,o,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,o,x,x,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,win
3,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,b,b,b,b,b,o,x,o,x,b,b,win
4,x,x,o,o,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,x,x,b,b,b,b,b,b,b,b,b,win


time: 71.1 ms (started: 2021-03-15 13:06:19 +00:00)


In [ ]:
# View the value counts, column count, and data types
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50667 entries, 0 to 50666
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a1      50667 non-null  object
 1   a2      50667 non-null  object
 2   a3      50667 non-null  object
 3   a4      50667 non-null  object
 4   a5      50667 non-null  object
 5   a6      50667 non-null  object
 6   b1      50667 non-null  object
 7   b2      50667 non-null  object
 8   b3      50667 non-null  object
 9   b4      50667 non-null  object
 10  b5      50667 non-null  object
 11  b6      50667 non-null  object
 12  c1      50667 non-null  object
 13  c2      50667 non-null  object
 14  c3      50667 non-null  object
 15  c4      50667 non-null  object
 16  c5      50667 non-null  object
 17  c6      50667 non-null  object
 18  d1      50667 non-null  object
 19  d2      50667 non-null  object
 20  d3      50667 non-null  object
 21  d4      50667 non-null  object
 22  d5      50667 non-null

In [ ]:
# Check whether the values contained in the training data are as expected
train_data.describe()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
count,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667,50667
unique,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
top,b,b,b,b,b,b,x,b,b,b,b,b,x,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,win
freq,18767,32565,41536,46256,48946,50281,19412,30835,40750,45936,48918,50294,24006,28230,39629,45566,48844,50277,21783,35253,43402,47283,49496,50427,30717,41642,46772,49073,50176,50599,25654,38385,45309,48643,50117,50602,22233,36024,44148,48176,50040,50606,33354


time: 436 ms (started: 2021-03-15 13:06:19 +00:00)


In [ ]:
# Check for possible class imbalance in the training data
train_data['class'].value_counts(normalize=True)

# The class imbalance is acceptable
# 'Win' has the highest frequency, followed by 'loss' and 'draw'

win     0.658298
loss    0.246235
draw    0.095466
Name: class, dtype: float64

time: 20.1 ms (started: 2021-03-15 13:06:20 +00:00)


## Data Preprocessing

In [ ]:
# This library will be used to transform the class labels to numerical labels
from sklearn.preprocessing import LabelEncoder

time: 735 ms (started: 2021-03-15 13:06:20 +00:00)


In [ ]:
# Instantiate the encoder
le = LabelEncoder()

time: 1.66 ms (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
# Encode the class labels
train_data['class'] = le.fit_transform(train_data['class'])

time: 38.5 ms (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
# Check whether the labels were encoded
train_data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,b,b,b,b,b,b,o,o,x,b,b,b,x,x,o,x,b,b,b,b,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,1
1,b,b,b,b,b,b,o,b,b,b,b,b,x,o,o,b,b,b,x,b,b,b,b,b,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,1
2,x,x,o,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,o,x,x,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,2
3,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,b,b,b,b,b,o,x,o,x,b,b,2
4,x,x,o,o,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,x,x,b,b,b,b,b,b,b,b,b,2


time: 91.9 ms (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
len(train_data.columns)

43

time: 3.92 ms (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
non_dummy_cols = ['class'] 
# Takes all 47 other columns
dummy_cols = list(set(train_data.columns) - set(non_dummy_cols))
train_data = pd.get_dummies(train_data, columns=dummy_cols)

train_data.head()

,class,e3_b,e3_o,e3_x,f5_b,f5_o,f5_x,g3_b,g3_o,g3_x,c2_b,c2_o,c2_x,a2_b,a2_o,a2_x,g5_b,g5_o,g5_x,g4_b,g4_o,g4_x,b3_b,b3_o,b3_x,d2_b,d2_o,d2_x,g6_b,g6_o,g6_x,d3_b,d3_o,d3_x,a5_b,a5_o,a5_x,c3_b,c3_o,c3_x,...,c4_x,e6_b,e6_o,e6_x,c5_b,c5_o,c5_x,g1_b,g1_o,g1_x,d5_b,d5_o,d5_x,f6_b,f6_o,f6_x,e1_b,e1_o,e1_x,b4_b,b4_o,b4_x,c1_b,c1_o,c1_x,e4_b,e4_o,e4_x,f3_b,f3_o,f3_x,e2_b,e2_o,e2_x,a4_b,a4_o,a4_x,b5_b,b5_o,b5_x
0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,...,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
1,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0
2,2,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
3,2,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
4,2,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0


time: 310 ms (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
len(train_data.columns)

127

time: 5.2 ms (started: 2021-03-15 13:06:21 +00:00)


## Model Building

In [ ]:
# This library contains a square root function that will allow us to easily compute for the train-validation split as done by Guyon (1997)
import math

time: 1.33 ms (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
# Compute for train-validation split based on the method of Guyon (1997)

val_split = (1 / math.sqrt(len(train_data.columns)-1))

train_split = 1 - val_split

print('Train-validation split:', train_split, '/', val_split)

Train-validation split: 0.9109129193625252 / 0.0890870806374748
time: 6.42 ms (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
from pycaret.classification import *

clf1 = setup(data = train_data, target = 'class', train_size = train_split, session_id = 42)

,Description,Value
0,session_id,42
1,Target,class
2,Target Type,Multiclass
3,Label Encoded,"0: 0, 1: 1, 2: 2"
4,Original Data,"(50667, 127)"
5,Missing Values,False
6,Numeric Features,126
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


time: 18.2 s (started: 2021-03-15 13:06:21 +00:00)


In [ ]:
top3 = compare_models(n_select = 3, sort = 'F1', include=['et', 'lightgbm', 'rf'])

# et, lightgbm, rf

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8248,0.9303,0.6146,0.8027,0.8011,0.6118,0.6248,10.608
lightgbm,Light Gradient Boosting Machine,0.8258,0.9350,0.6058,0.8041,0.7971,0.6126,0.6269,1.912
rf,Random Forest Classifier,0.8230,0.9302,0.6018,0.8030,0.7952,0.6015,0.6192,8.338


time: 3min 51s (started: 2021-03-15 13:06:45 +00:00)


In [ ]:
et = create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8245,0.9249,0.6119,0.8010,0.7998,0.6118,0.6245
1,0.8256,0.9339,0.6147,0.8041,0.8018,0.6125,0.6264
2,0.8330,0.9334,0.6340,0.8165,0.8125,0.6322,0.6439
3,0.8254,0.9308,0.6162,0.7995,0.8023,0.6169,0.6273
4,0.8228,0.9280,0.6040,0.7957,0.7966,0.6048,0.6194
5,0.8232,0.9286,0.6017,0.7951,0.7958,0.6056,0.6203
6,0.8238,0.9298,0.6127,0.8017,0.8001,0.6105,0.6227
7,0.8199,0.9283,0.6053,0.7976,0.7949,0.5978,0.6127
8,0.8256,0.9351,0.6278,0.8051,0.8057,0.6201,0.6289
9,0.8264,0.9303,0.6155,0.8018,0.8031,0.6158,0.6286


time: 2min 7s (started: 2021-03-15 07:37:00 +00:00)


In [ ]:
ensemble_et = ensemble_model(et, method = 'Bagging')

# Bagging generated a lower mean F1 score

NameError: ignored

In [ ]:
blender = blend_models(top3, optimize='F1')

# Blending generated a marginally lower F1 score

In [ ]:
stacker = stack_models(top3, optimize = 'F1', fold = 3)

# Stacking generated a higher F1 score

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8351,0.9415,0.6506,0.8164,0.8186,0.6486,0.6539
1,0.8376,0.9425,0.6522,0.8187,0.8212,0.6535,0.6589
2,0.8348,0.9416,0.6505,0.8164,0.8186,0.6478,0.6530
Mean,0.8358,0.9419,0.6511,0.8172,0.8195,0.6500,0.6553
SD,0.0013,0.0004,0.0008,0.0011,0.0012,0.0025,0.0026


time: 4min 35s (started: 2021-03-15 13:12:55 +00:00)


In [ ]:
tuned_stacker = tune_model(stacker, optimize = 'F1', n_iter = 3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8354,0.9399,0.6533,0.8167,0.8198,0.6509,0.6555
1,0.8458,0.9503,0.6622,0.8277,0.8297,0.6699,0.6759
2,0.8494,0.9492,0.6826,0.8358,0.8369,0.6815,0.6857
3,0.8470,0.9483,0.6668,0.8285,0.8321,0.6760,0.6805
4,0.8442,0.9467,0.6662,0.8259,0.8298,0.6696,0.6741
5,0.8462,0.9464,0.6669,0.8283,0.8317,0.6727,0.6775
6,0.8373,0.9461,0.6518,0.8178,0.8210,0.6534,0.6585
7,0.8407,0.9461,0.6626,0.8241,0.8259,0.6610,0.6659
8,0.8427,0.9485,0.6750,0.8281,0.8305,0.6694,0.6728
9,0.8392,0.9455,0.6600,0.8215,0.8245,0.6580,0.6628


time: 53min 48s (started: 2021-03-15 13:17:31 +00:00)


In [ ]:
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/connect-4/test_set_label.csv')

time: 288 ms (started: 2021-03-15 05:45:07 +00:00)


In [ ]:
test_data = pd.get_dummies(test_data)

test_data.head()

,a1_b,a1_o,a1_x,a2_b,a2_o,a2_x,a3_b,a3_o,a3_x,a4_b,a4_o,a4_x,a5_b,a5_o,a5_x,a6_b,a6_o,a6_x,b1_b,b1_o,b1_x,b2_b,b2_o,b2_x,b3_b,b3_o,b3_x,b4_b,b4_o,b4_x,b5_b,b5_o,b5_x,b6_b,b6_o,b6_x,c1_b,c1_o,c1_x,c2_b,...,e5_x,e6_b,e6_o,e6_x,f1_b,f1_o,f1_x,f2_b,f2_o,f2_x,f3_b,f3_o,f3_x,f4_b,f4_o,f4_x,f5_b,f5_o,f5_x,f6_b,f6_o,f6_x,g1_b,g1_o,g1_x,g2_b,g2_o,g2_x,g3_b,g3_o,g3_x,g4_b,g4_o,g4_x,g5_b,g5_o,g5_x,g6_b,g6_o,g6_x
0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0
1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1,1,...,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
2,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
3,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0
4,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0


time: 155 ms (started: 2021-03-15 05:46:23 +00:00)


In [ ]:
pred = predict_model(stacker, data = test_data)

time: 2.54 s (started: 2021-03-15 05:46:37 +00:00)


In [ ]:
pred

,a1_b,a1_o,a1_x,a2_b,a2_o,a2_x,a3_b,a3_o,a3_x,a4_b,a4_o,a4_x,a5_b,a5_o,a5_x,a6_b,a6_o,a6_x,b1_b,b1_o,b1_x,b2_b,b2_o,b2_x,b3_b,b3_o,b3_x,b4_b,b4_o,b4_x,b5_b,b5_o,b5_x,b6_b,b6_o,b6_x,c1_b,c1_o,c1_x,c2_b,...,e6_o,e6_x,f1_b,f1_o,f1_x,f2_b,f2_o,f2_x,f3_b,f3_o,f3_x,f4_b,f4_o,f4_x,f5_b,f5_o,f5_x,f6_b,f6_o,f6_x,g1_b,g1_o,g1_x,g2_b,g2_o,g2_x,g3_b,g3_o,g3_x,g4_b,g4_o,g4_x,g5_b,g5_o,g5_x,g6_b,g6_o,g6_x,Label,Score
0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0.9825
1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1,1,...,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0.6824
2,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,2,0.9526
3,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,2,0.6103
4,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0.6747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16885,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0.9118
16886,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,2,0.9437
16887,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,2,0.9846
16888,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,...,0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,2,0.9373


time: 98.9 ms (started: 2021-03-15 05:46:48 +00:00)


In [ ]:
target = pd.DataFrame(pred['Label'])

time: 2.23 ms (started: 2021-03-15 05:48:42 +00:00)


In [ ]:
target

,Label
0,1
1,1
2,2
3,2
4,0
...,...
16885,1
16886,2
16887,2
16888,2


time: 10.6 ms (started: 2021-03-15 05:48:44 +00:00)


In [ ]:
target = le.inverse_transform(target)

time: 2.52 ms (started: 2021-03-15 05:49:30 +00:00)


In [ ]:
target

array(['loss', 'loss', 'win', ..., 'win', 'win', 'win'], dtype=object)

time: 6.45 ms (started: 2021-03-15 05:49:33 +00:00)


In [ ]:
target = pd.DataFrame(target, columns=['prediction'])

time: 1.73 ms (started: 2021-03-15 05:51:25 +00:00)


In [ ]:
target

,prediction
0,loss
1,loss
2,win
3,win
4,draw
...,...
16885,loss
16886,win
16887,win
16888,win


time: 13.7 ms (started: 2021-03-15 05:51:31 +00:00)


In [ ]:
target['prediction'].to_csv('sprint 27 submission 2.csv', index=False)

from google.colab import files
files.download('sprint 27 submission 2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 27 ms (started: 2021-03-15 05:54:26 +00:00)
